In [1]:
import os
import json
from collections import Counter
from itertools import chain
import json
import gc

BioASQ_training_file = "BioASQ-trainingDataset6b.json"

path = os.path.join('..','data',BioASQ_training_file)

## Load the bioasq training data

Show some statistics

In [2]:
data = json.load(open(path))["questions"]

print("Number of questions: ",len(data))

#count questions by type
print(Counter([ q["type"] for q in data ]))

Number of questions:  2251
Counter({'factoid': 619, 'yesno': 616, 'summary': 531, 'list': 485})


## Prepare some data for document retriavel 

In [3]:
#gather the revelent pmid of the documents
question_document_r = [(q["body"],list(map(lambda x: x.split("/")[-1],q["documents"]))) for q in data]
     


#only relevant documents pmid
doc_y_pmid = list(chain.from_iterable([map(lambda x:int(x),d) for (q,d) in question_document_r]))

#find min max pmid
print("Min pmid:",min(doc_y_pmid),"Max pmid:",max(doc_y_pmid))

Min pmid: 9797 Max pmid: 27957702


## Load pubmed xml file

In [4]:

READ=False #else LOAD

if READ:
    import pubmed_parser as pp

    path = os.path.join('..','data')
    path_xml = pp.list_xml_path(path) 

    pubmed_doc_list = []

    for p in path_xml:
        print("read",p,end="\r")
        parsed = pp.parse_medline_xml(p)
        parsed_abstract = [{"title":a["title"],"abstract":a["abstract"],"pmid":int(a["pmid"])} for a in parsed if a["abstract"]!=""]

        del parsed

        for x in parsed_abstract:
            pubmed_doc_list.append(x)

        del parsed_abstract                                                                           
        
        #call garbage collector
        gc.collect()
    print()
else:
    with open('data.json', 'r') as infile:
        pubmed_doc_list = json.load(infile)


#call garbage collector
gc.collect()        

print("Number articles",len(pubmed_doc_list))

read ..\data\pubmed18n0809.xml
Number articles 1168777


In [5]:
## save

gc.collect()


with open('data.json', 'w') as outfile:
    json.dump(pubmed_doc_list, outfile)
    


In [11]:


gc.collect()

%whos

Variable               Type      Data/Info
------------------------------------------
BioASQ_training_file   str       BioASQ-trainingDataset6b.json
Counter                type      <class 'collections.Counter'>
READ                   bool      True
chain                  type      <class 'itertools.chain'>
data                   list      n=2251
doc_x_pmid             list      n=1168777
doc_y_pmid             list      n=27034
gc                     module    <module 'gc' (built-in)>
json                   module    <module 'json' from 'c:\\<...>\lib\\json\\__init__.py'>
os                     module    <module 'os' from 'c:\\us<...>\\envs\\bio\\lib\\os.py'>
p                      str       ..\data\pubmed18n0809.xml
path                   str       ..\data
path_xml               list      n=60
pp                     module    <module 'pubmed_parser' f<...>med_parser\\__init__.py'>
pubmed_doc_list        list      n=1168777
question_document_r    list      n=2251
x                    

In [7]:
print("Number articles",len(pubmed_doc_list))

print("Select random article\n", pubmed_doc_list[42])

doc_x_pmid = [int(doc["pmid"]) for doc in pubmed_doc_list]

#find min max pmid
print("\nMin pmid:",min(doc_x_pmid),"Max pmid:",max(doc_x_pmid))


Number articles 1168777
Select random article
 {'title': 'Behavior of soluble and immobilized acid phosphatase in hydro-organic media.', 'abstract': 'The hydrolysis of p-nitrophenyl phosphate by wheat germ acid phosphatase (orthophosphoric monoester phosphohydrolase, EC 3.1.3.2) has been investigated in mixtures of aqueous buffers with acetone, dioxane and acetonitrile. The enzyme was either in free solution or immobilized on a pellicular support which consisted of a porous carbonaceous layer on solid glass beads. The highest enzyme activity was obtained in acetone and acetonitrile mixed with citrate buffer over a wide range of organic solvent concentration. In 50% (v/v) acetone both V and Km of the immobilized enzyme were about half of the values in the neat aqueous buffer, but the Ki for inorganic phosphate was unchanged. In 50% (v/v) mixtures of various solvents and citrate buffers of different pH, the enzymic activity was found to depend on the pH of the aqueous buffer component ra

## Cross x data with y data

In [8]:
print("#Y",len(doc_y_pmid))
print("#X",len(doc_x_pmid))

y_intersect_x = list(set(doc_y_pmid)&set(doc_x_pmid))
print(len(y_intersect_x))
print("% of Y coverege in X",len(y_intersect_x)/len(doc_y_pmid))

#Y 27034
#X 1168777
1777
% of Y coverege in X 0.06573204113338758
